In [19]:
from utils import MongoConnectionManager
from pprint import pprint
import json
import os

In [2]:
def get_dependencies(section_name):
    with open(f'static/{section_name}_dependencies.json', 'r') as reader:
        filters = json.loads(reader.read())
    
    fields = []
    for key, value in filters.items():
        if value['type'] == 'select' and value['input']:
            fields.append(value['item_id'])
    
    field_names = {}
    for field in fields:
        field_names[field] = list(set(fields) - set([field]))
    
    return field_names

In [12]:
def facet_query_builder(section_name):
    fields = get_dependencies(section_name)
    facet_query = {}
    for field, dependents in fields.items():
        field_name = field.upper()
        match_query = {field_name: {'$ne': None}}
        group_query = {'_id': f'${field_name}'}
        for dep in dependents:
            dep_field = dep.upper()
            group_query[dep] = {
                '$addToSet': {
                    '$cond': {
                        'if': {'$ne': [f'${dep_field}', None]},
                        'then': f'${dep_field}',
                        'else': '$$REMOVE'
                    }
                }
            }
        facet_query[field] = [
            {'$match': match_query},
            {'$group': group_query}
        ]
    return facet_query

In [13]:
def get_filter_data(section_name):
    pipeline = [{'$facet': facet_query_builder(section_name)}]
    database = os.environ.get('DATABASE')
    with MongoConnectionManager(database, f'{section_name}_collection') as collection:
        data = list(collection.aggregate(pipeline=pipeline, allowDiskUse=True))[0]
    return data

In [14]:
def prepare_filter_data(data):
    filter_data = {}
    for key, value in data.items():
        item_data = {}
        for item in value:
            item_id = item['_id']
            item_data[item_id] = {k: v for k, v in item.items()}
            del item_data[item_id]['_id']
        filter_data[key] = item_data
    return filter_data

In [15]:
section_name = os.environ.get('SECTION')
data = get_filter_data(section_name)
filter_data = prepare_filter_data(data)

In [ ]:
pprint(filter_data)